In [8]:
import torch
import numpy as np
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms

from torch import nn
from torch import optim
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
#from torch
from torchsummary import summary

## Обучите CNN  с аугментацией данных.

In [2]:
pip install torchsummary

Note: you may need to restart the kernel to use updated packages.


In [6]:
pip install albumentations

Note: you may need to restart the kernel to use updated packages.


In [10]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [15]:
def train_valid_split(Xt):
    X_train, X_test = train_test_split(Xt, test_size=0.05, random_state=13)
    return X_train, X_test

In [12]:
dataset = torchvision.datasets.CIFAR100(root='data/', train=True, download=True)


class MyOwnCifar(torch.utils.data.Dataset):

    def __init__(self, init_dataset, transform=None):
        self._base_dataset = init_dataset
        self.transform = transform

    def __len__(self):
        return len(self._base_dataset)

    def __getitem__(self, idx):
        img = self._base_dataset[idx][0]
        if self.transform is not None:
            img = self.transform(img)
        return img, self._base_dataset[idx][1]

100%|██████████| 169001437/169001437 [00:02<00:00, 74227968.87it/s]


Extracting data/cifar-100-python.tar.gz to data/


## Аргументация

In [13]:
import albumentations as alb
#from albumentations.pytorch import ToTensor
import numpy as np


alb_transform = alb.Compose(
    [
        #alb.Resize(height=256, width=256),
        #alb.RandomCrop(height=224, width=224),
        #alb.HorizontalFlip(p=0.25, ),
        #alb.
        #alb.SmallestMaxSize(max_size=160,always_apply=False),
        alb.ShiftScaleRotate(shift_limit=0.05, scale_limit=0.05, rotate_limit=10, p=1, always_apply=False),
        #alb.RGBShift(r_shift_limit=15, g_shift_limit=15, b_shift_limit=15, p=0.5, always_apply=False),
        alb.RandomBrightnessContrast(p=0.15, always_apply=False),
        #alb.OneOf([
         #   alb.MotionBlur(p=1),
         #   alb.OpticalDistortion(p=1),
         #   alb.GaussNoise(p=1)
        #], p=1),
        alb.Normalize(mean=(0.485, 0.456, 0.406),
                      std=(0.229, 0.224, 0.225)),
         #ToTensor()
    ])

In [16]:
trans_actions = transforms.Compose([alb_transform,
                                    #transforms.RandomCrop(224, padding=4),
                                    transforms.ToTensor(),
                                    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                                         std=[0.229, 0.224, 0.225])])
#mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]
valid_transforms = transforms.Compose([transforms.ToTensor(),
                                       transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                                         std=[0.229, 0.224, 0.225])])

train_dataset, valid_dataset = train_valid_split(dataset)

train_dataset = MyOwnCifar(train_dataset, trans_actions)
valid_dataset = MyOwnCifar(valid_dataset, valid_transforms)

train_loader = torch.utils.data.DataLoader(train_dataset,
                          batch_size=700,
                          shuffle=True,
                          num_workers=1)

valid_loader = torch.utils.data.DataLoader(valid_dataset,
                          batch_size=700,
                          shuffle=False,
                          num_workers=1)

## Модель

In [17]:
class Net_CCN_5(nn.Module):

    def __init__(self):
        super(Net_CCN_5, self).__init__()

        self.dp1 = nn.Dropout(0.3)
        self.bn1 = torch.nn.BatchNorm2d(3)
        self.conv1 = torch.nn.Conv2d(3, 30, 3)#32

        self.bn2 = torch.nn.BatchNorm2d(30)
        self.conv2 = torch.nn.Conv2d(30, 64, 3) #32
        self.dp2 = nn.Dropout(0.3)

        self.bn3= torch.nn.BatchNorm2d(64)
        self.conv3 = torch.nn.Conv2d(64, 128, 3) #32 вход
        self.dp3 = nn.Dropout(0.2)
        #polling запулингировали 16

        self.bn4 = torch.nn.BatchNorm2d(128)
        self.conv4 = torch.nn.Conv2d(128, 128, 3) #16 в 14
        self.dp4 = nn.Dropout(0.2)

        self.bn5= torch.nn.BatchNorm2d(128)
        self.conv5 = torch.nn.Conv2d(128, 256, 3) #14 в 12
        self.dp5 = nn.Dropout(0.2)

        #polling #из 12 в 6

        self.bn6 = torch.nn.BatchNorm2d(256)
        self.conv6 = torch.nn.Conv2d(256, 256, 3) #из 6 в 4
        self.dp6 = nn.Dropout(0.1)

        self.bn7 = torch.nn.BatchNorm2d(256)
        self.conv7 = torch.nn.Conv2d(256, 512, 3) # 2
        #self.fl = torch.nn.Flatten(2)
        self.dp7 = nn.Dropout(0.1)

        self.bn8 = torch.nn.BatchNorm2d(512)

        self.fc1 = torch.nn.Linear(2048, 1024) #256 на 3 на 3
        self.dp_fc1 = nn.Dropout(0.3)
        self.fc2 = torch.nn.Linear(1024, 512)
        self.dp_fc2 = nn.Dropout(0.2)
        self.out = torch.nn.Linear(512, 100)

    def forward(self, x):

        x = self.dp1(x)
        x = self.bn1(x) # 32 на 32
        x = torch.nn.functional.pad(x, (1, 1, 1, 1), mode='reflect')
        x = self.conv1(x) # 32 на 32 падинг 1
        x = F.relu(x)

        x = self.bn2(x)
        x = torch.nn.functional.pad(x, (1, 1, 1, 1), mode='reflect')
        x = self.conv2(x) #30 на 30
        x = self.dp2(x)
        x = F.relu(x)

        x = self.bn3(x)
        x = torch.nn.functional.pad(x, (1, 1, 1, 1), mode='reflect')
        x = self.conv3(x) #28 на 28
        x = self.dp3(x)
        x = F.relu(x)

        x = F.avg_pool2d(x,2,stride=2) #14 на 14

        x = self.bn4(x) #14 на 14
        x = self.conv4(x) #12 на 12
        #x = self.dp4(x)
        x = F.relu(x)

        x = self.bn5(x)
        x = self.conv5(x) #
        #x = self.dp5(x)
        x = F.relu(x)

        x = F.avg_pool2d(x,2,stride=2)

        x = self.bn6(x)
        x = self.conv6(x)
        #x = self.dp6(x)
        x = F.relu(x)

        x = self.bn7(x)
        x = self.conv7(x)
        #x = self.dp7(x)
        x = F.relu(x)
        #x = self.fl(x)
        x = self.bn8(x)

        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        x = self.dp_fc1(x)
        x = F.relu(x)

        x = self.fc2(x)
        x = self.dp_fc2(x)
        x = F.relu(x)

        return self.out(x)

net = Net_CCN_5().to(device)
#print(net)

from torchsummary import summary

summary(net, input_size=(3, 32, 32))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
           Dropout-1            [-1, 3, 32, 32]               0
       BatchNorm2d-2            [-1, 3, 32, 32]               6
            Conv2d-3           [-1, 30, 32, 32]             840
       BatchNorm2d-4           [-1, 30, 32, 32]              60
            Conv2d-5           [-1, 64, 32, 32]          17,344
           Dropout-6           [-1, 64, 32, 32]               0
       BatchNorm2d-7           [-1, 64, 32, 32]             128
            Conv2d-8          [-1, 128, 32, 32]          73,856
           Dropout-9          [-1, 128, 32, 32]               0
      BatchNorm2d-10          [-1, 128, 16, 16]             256
           Conv2d-11          [-1, 128, 14, 14]         147,584
      BatchNorm2d-12          [-1, 128, 14, 14]             256
           Conv2d-13          [-1, 256, 12, 12]         295,168
      BatchNorm2d-14            [-1, 25

In [18]:
torch.cuda.empty_cache()

## Обучение

In [19]:
optimizer = torch.optim.NAdam(net.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

In [20]:
num_epochs = 100
net.train()

for epoch in range(num_epochs):
    running_loss, running_items, running_right = 0.0, 0.0, 0.0
    for i, data in enumerate(valid_loader):
        inputs, labels = data[0].to(device), data[1].to(device)

        # обнуляем градиент
        optimizer.zero_grad()

        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # выводим статистику о процессе обучения
        running_loss += loss.item()
        running_items += len(labels)
        running_right += (labels == torch.max(outputs, 1)[1]).sum()

        # выводим статистику о процессе обучения
        if i % 300 == 0:    # печатаем каждые 300 mini-batches
            net.eval()

            print(f'Epoch [{epoch + 1}/{num_epochs}]. ' \
                  f'Step [{i + 1}/{len(train_loader)}]. ' \
                  f'Loss: {running_loss / running_items:.4f}. ' \
                  f'Acc: {(running_right / running_items)*100:.4f}', end='. ')
            running_loss, running_items, running_right = 0.0, 0.0, 0.0

            test_running_right, test_running_total = 0.0, 0.0
            for i, data in enumerate(valid_loader):

                test_outputs = net(data[0].to(device))
                test_running_total += len(data[1])
                test_running_right += (data[1].to(device) == torch.max(test_outputs, 1)[1]).sum()

            print(f'Test acc: {(test_running_right / test_running_total)*100:.3f}')

        net.train()

print('Training is finished!')

Epoch [1/100]. Step [1/68]. Loss: 0.0066. Acc: 1.1429. Test acc: 1.160
Epoch [2/100]. Step [1/68]. Loss: 0.0063. Acc: 5.1429. Test acc: 1.920
Epoch [3/100]. Step [1/68]. Loss: 0.0062. Acc: 5.5714. Test acc: 1.280
Epoch [4/100]. Step [1/68]. Loss: 0.0060. Acc: 6.4286. Test acc: 4.560
Epoch [5/100]. Step [1/68]. Loss: 0.0059. Acc: 5.8571. Test acc: 7.680
Epoch [6/100]. Step [1/68]. Loss: 0.0059. Acc: 8.2857. Test acc: 6.680
Epoch [7/100]. Step [1/68]. Loss: 0.0056. Acc: 8.8571. Test acc: 12.360
Epoch [8/100]. Step [1/68]. Loss: 0.0055. Acc: 11.4286. Test acc: 13.040
Epoch [9/100]. Step [1/68]. Loss: 0.0051. Acc: 13.7143. Test acc: 12.360
Epoch [10/100]. Step [1/68]. Loss: 0.0051. Acc: 14.1429. Test acc: 11.480
Epoch [11/100]. Step [1/68]. Loss: 0.0048. Acc: 15.2857. Test acc: 15.080
Epoch [12/100]. Step [1/68]. Loss: 0.0049. Acc: 17.5714. Test acc: 17.080
Epoch [13/100]. Step [1/68]. Loss: 0.0047. Acc: 17.4286. Test acc: 15.960
Epoch [14/100]. Step [1/68]. Loss: 0.0043. Acc: 22.4286. Tes

In [21]:
net.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in valid_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('The final Test Accuracy of the model: {} %'.format((correct / total) * 100))


The final Test Accuracy of the model: 90.72 %


## Выводы

Аргументация в целом позволяет снизить переобучение, но необходимо правильная, оптимальная настройков параметров   
если параметры аргументации будут генерировать изображения которые мало похожи на изображения в тестовой выборке пользы от аргументации не будет.  
В моем случае аргументация вместе с подбором % dropout и скорости обучения так же позволила ускорить обучение и снизить переобучение.  
Для данной задачи достаточно 40-50 эпох обучения

Созданая модель хоть и приходит к оптимуму за большее количество эпох по тренировочной выборке, 
но приходить туда все таки быстрее по времени за счет меньшего количества параметров сети которые необходимо пересчитывать (веса)